In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from glob import glob
from sklearn.utils import shuffle
from scipy.fftpack import fft
from sklearn.preprocessing import StandardScaler
from scipy.stats import entropy
from math import log, e

In [2]:
def SS(df):
    X = df['activity'].values
    X = X.reshape((len(X), 1))
    X = StandardScaler().fit_transform(X)
    df['activity'] = X
    return(df)

In [3]:
def abs_energy(x):
    """
    Returns the absolute energy of the time series which is the sum over the squared values

    .. math::

        E = \\sum_{i=1,\ldots, n} x_i^2

    :param x: the time series to calculate the feature of
    :type x: numpy.ndarray
    :return: the value of this feature
    :return type: float
    """
    if not isinstance(x, (np.ndarray, pd.Series)):
        x = np.asarray(x)
    return np.dot(x, x)

In [4]:
def FetureExtraction(df,Filename):
 fdf = fft(df['activity'])
 df["activity"] = fdf.real
 df = SS(df)
 df = df['activity'].resample('60T')
 mean = df.mean()
 std = df.std()
 var = df.var()
 kurtosis = df.apply(pd.DataFrame.kurt)
 skewness = df.apply(pd.DataFrame.skew)
 quantile1 = df.quantile(q=0.01)
 quantile2 = df.quantile(q=0.05)
 quantile3 = df.quantile(q=0.25)
 quantile4 = df.quantile(q=0.75)
 quantile5 = df.quantile(q=0.95)
 quantile6 = df.quantile(q=0.99)
 trimed_Mean = df.apply(stats.trim_mean, .1)
 cofvar = std/ mean
 invcofvar = mean/std
 minimum = df.min()
 maxmimum = df.max()
 median = df.median()
 absenergy = df.apply(abs_energy)
 IQR = df.apply(stats.iqr, interpolation = 'midpoint') 
 frames = [mean,std,var,kurtosis,skewness,quantile1,quantile2,quantile3,quantile4,quantile5,quantile6,trimed_Mean,cofvar,invcofvar,minimum,maxmimum,median,absenergy,IQR]
 final_data = pd.concat(frames,axis=1, sort=False)
 final_data.columns = ['fmean','fstd','fvar','fkurtosis','fskewness','fquantile1','fquantile2','fquantile3','fquantile4','fquantile5','fquantile6','ftrimed_Mean','fcofvar','finvcofvar','fminimum','fmaxmimum','fmedian','fabsenergy','fIQR']
 #final_data['Class'] = condition_1
 condition = Filename
 final_data.insert(loc=0, column='FId', value=condition)
 return(final_data)

In [5]:
stock_files_condition = sorted(glob('./depresjon-dataset/condition/condition_*.csv'))
stock_files_condition

['./depresjon-dataset/condition\\condition_1.csv',
 './depresjon-dataset/condition\\condition_10.csv',
 './depresjon-dataset/condition\\condition_11.csv',
 './depresjon-dataset/condition\\condition_12.csv',
 './depresjon-dataset/condition\\condition_13.csv',
 './depresjon-dataset/condition\\condition_14.csv',
 './depresjon-dataset/condition\\condition_15.csv',
 './depresjon-dataset/condition\\condition_16.csv',
 './depresjon-dataset/condition\\condition_17.csv',
 './depresjon-dataset/condition\\condition_18.csv',
 './depresjon-dataset/condition\\condition_19.csv',
 './depresjon-dataset/condition\\condition_2.csv',
 './depresjon-dataset/condition\\condition_20.csv',
 './depresjon-dataset/condition\\condition_21.csv',
 './depresjon-dataset/condition\\condition_22.csv',
 './depresjon-dataset/condition\\condition_23.csv',
 './depresjon-dataset/condition\\condition_3.csv',
 './depresjon-dataset/condition\\condition_4.csv',
 './depresjon-dataset/condition\\condition_5.csv',
 './depresjon-dat

In [6]:
li = []
for filename in stock_files_condition:
    #condtion = 'condition_'+ filename
    df = pd.read_csv(filename, parse_dates=["timestamp"], index_col="timestamp")
    li.append(df)

In [7]:
print(li[0])

                           date  activity
timestamp                                
2003-05-07 12:00:00  2003-05-07         0
2003-05-07 12:01:00  2003-05-07       143
2003-05-07 12:02:00  2003-05-07         0
2003-05-07 12:03:00  2003-05-07        20
2003-05-07 12:04:00  2003-05-07       166
...                         ...       ...
2003-05-23 15:19:00  2003-05-23         0
2003-05-23 15:20:00  2003-05-23         0
2003-05-23 15:21:00  2003-05-23         0
2003-05-23 15:22:00  2003-05-23         0
2003-05-23 15:23:00  2003-05-23       533

[23244 rows x 2 columns]


In [8]:
print(len(li[0]))

23244


In [9]:
Final_Data1 =[]
#condtion = 'condition_'
for i, df in zip(range(1,len(li)+1),li):
 condtion = 'condition_'+str(i)
 Final_Data1.append(FetureExtraction(df,condtion))

In [10]:
Final_Data1

[                             FId     fmean       fstd        fvar  fkurtosis  \
 timestamp                                                                      
 2003-05-07 12:00:00  condition_1  1.462541  13.286953  176.543115  30.587756   
 2003-05-07 13:00:00  condition_1  0.117842   3.158199    9.974221  -0.080292   
 2003-05-07 14:00:00  condition_1  0.172950   2.183673    4.768428   0.638762   
 2003-05-07 15:00:00  condition_1 -0.000085   1.681192    2.826405  -0.473312   
 2003-05-07 16:00:00  condition_1 -0.099009   1.515064    2.295417  -0.033818   
 ...                          ...       ...        ...         ...        ...   
 2003-05-23 11:00:00  condition_1  0.081929   1.308398    1.711905  -0.263469   
 2003-05-23 12:00:00  condition_1  0.051324   1.845223    3.404847  -0.116965   
 2003-05-23 13:00:00  condition_1  0.055715   2.742776    7.522818  -0.353311   
 2003-05-23 14:00:00  condition_1  0.089069   3.395893   11.532092  -0.366217   
 2003-05-23 15:00:00  condit

In [11]:
len(Final_Data1)

23

In [12]:
final_data1 = pd.concat(Final_Data1,axis=0)
final_data1

,FId,fmean,fstd,fvar,fkurtosis,fskewness,fquantile1,fquantile2,fquantile3,fquantile4,fquantile5,fquantile6,ftrimed_Mean,fcofvar,finvcofvar,fminimum,fmaxmimum,fmedian,fabsenergy,fIQR
timestamp,,,,,,,,,,,,,,,,,,,,
2003-05-07 12:00:00,condition_1,1.462541,13.286953,176.543115,30.587756,5.113729,-11.798675,-8.282023,-3.050700,3.105759,6.339978,61.352314,-0.302254,9.084842,0.110073,-15.124664,86.675226,-0.943192,10544.385352,6.309015
2003-05-07 13:00:00,condition_1,0.117842,3.158199,9.974221,-0.080292,-0.292813,-7.057130,-4.990460,-2.045783,2.218769,4.960632,5.991059,0.189661,26.800245,0.037313,-8.768296,6.758724,0.254860,589.312248,4.329223
2003-05-07 14:00:00,condition_1,0.172950,2.183673,4.768428,0.638762,0.036386,-4.454137,-3.851889,-1.063322,1.589911,3.238580,5.828336,0.205902,12.626041,0.079201,-4.520817,6.185963,0.195132,283.131965,2.677316
2003-05-07 15:00:00,condition_1,-0.000085,1.681192,2.826405,-0.473312,0.032258,-3.383892,-2.613928,-1.159312,1.173533,2.597093,3.387807,-0.000755,-19667.292944,-0.000051,-3.857517,3.718487,0.033770,166.757919,2.394075
2003-05-07 16:00:00,condition_1,-0.099009,1.515064,2.295417,-0.033818,0.007128,-3.496541,-2.345351,-1.200489,0.834629,2.494810,3.078181,-0.115383,-15.302297,-0.065350,-3.905108,3.329462,-0.056935,136.017793,2.040137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004-06-10 11:00:00,condition_23,-0.248305,1.480854,2.192929,0.253270,0.152783,-3.566330,-2.561179,-0.981733,0.650651,2.055527,3.119941,-0.265890,-5.963842,-0.167677,-3.637771,3.304007,-0.480968,133.082162,1.705071
2004-06-10 12:00:00,condition_23,0.252677,2.434540,5.926984,1.669970,-0.570104,-6.840116,-3.387387,-1.031348,1.992138,3.609333,5.320898,0.347784,9.634999,0.103788,-7.311436,6.276504,0.236197,353.522815,3.070362
2004-06-10 13:00:00,condition_23,-0.938612,2.871737,8.246871,-0.733369,0.171609,-6.186788,-5.241718,-3.102307,1.362081,3.468978,5.187083,-0.992230,-3.059558,-0.326845,-6.652778,5.217019,-0.923367,539.424863,4.513501


In [13]:
final_data1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9214 entries, 2003-05-07 12:00:00 to 2004-06-10 15:00:00
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   FId           9214 non-null   object 
 1   fmean         9214 non-null   float64
 2   fstd          9213 non-null   float64
 3   fvar          9213 non-null   float64
 4   fkurtosis     9212 non-null   float64
 5   fskewness     9212 non-null   float64
 6   fquantile1    9214 non-null   float64
 7   fquantile2    9214 non-null   float64
 8   fquantile3    9214 non-null   float64
 9   fquantile4    9214 non-null   float64
 10  fquantile5    9214 non-null   float64
 11  fquantile6    9214 non-null   float64
 12  ftrimed_Mean  9214 non-null   float64
 13  fcofvar       9213 non-null   float64
 14  finvcofvar    9213 non-null   float64
 15  fminimum      9214 non-null   float64
 16  fmaxmimum     9214 non-null   float64
 17  fmedian       9214 non-null   float

In [14]:
final_data1.insert(loc=1, column='Fclass', value='1')
final_data1.insert(loc=2, column='Fclass_str', value='c1')

In [15]:
final_data1

,FId,Fclass,Fclass_str,fmean,fstd,fvar,fkurtosis,fskewness,fquantile1,fquantile2,...,fquantile5,fquantile6,ftrimed_Mean,fcofvar,finvcofvar,fminimum,fmaxmimum,fmedian,fabsenergy,fIQR
timestamp,,,,,,,,,,,,,,,,,,,,,
2003-05-07 12:00:00,condition_1,1,c1,1.462541,13.286953,176.543115,30.587756,5.113729,-11.798675,-8.282023,...,6.339978,61.352314,-0.302254,9.084842,0.110073,-15.124664,86.675226,-0.943192,10544.385352,6.309015
2003-05-07 13:00:00,condition_1,1,c1,0.117842,3.158199,9.974221,-0.080292,-0.292813,-7.057130,-4.990460,...,4.960632,5.991059,0.189661,26.800245,0.037313,-8.768296,6.758724,0.254860,589.312248,4.329223
2003-05-07 14:00:00,condition_1,1,c1,0.172950,2.183673,4.768428,0.638762,0.036386,-4.454137,-3.851889,...,3.238580,5.828336,0.205902,12.626041,0.079201,-4.520817,6.185963,0.195132,283.131965,2.677316
2003-05-07 15:00:00,condition_1,1,c1,-0.000085,1.681192,2.826405,-0.473312,0.032258,-3.383892,-2.613928,...,2.597093,3.387807,-0.000755,-19667.292944,-0.000051,-3.857517,3.718487,0.033770,166.757919,2.394075
2003-05-07 16:00:00,condition_1,1,c1,-0.099009,1.515064,2.295417,-0.033818,0.007128,-3.496541,-2.345351,...,2.494810,3.078181,-0.115383,-15.302297,-0.065350,-3.905108,3.329462,-0.056935,136.017793,2.040137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004-06-10 11:00:00,condition_23,1,c1,-0.248305,1.480854,2.192929,0.253270,0.152783,-3.566330,-2.561179,...,2.055527,3.119941,-0.265890,-5.963842,-0.167677,-3.637771,3.304007,-0.480968,133.082162,1.705071
2004-06-10 12:00:00,condition_23,1,c1,0.252677,2.434540,5.926984,1.669970,-0.570104,-6.840116,-3.387387,...,3.609333,5.320898,0.347784,9.634999,0.103788,-7.311436,6.276504,0.236197,353.522815,3.070362
2004-06-10 13:00:00,condition_23,1,c1,-0.938612,2.871737,8.246871,-0.733369,0.171609,-6.186788,-5.241718,...,3.468978,5.187083,-0.992230,-3.059558,-0.326845,-6.652778,5.217019,-0.923367,539.424863,4.513501


In [17]:
stock_files_condition = sorted(glob('./depresjon-dataset/control/control_*.csv'))
stock_files_condition

['./depresjon-dataset/control\\control_1.csv',
 './depresjon-dataset/control\\control_10.csv',
 './depresjon-dataset/control\\control_11.csv',
 './depresjon-dataset/control\\control_12.csv',
 './depresjon-dataset/control\\control_13.csv',
 './depresjon-dataset/control\\control_14.csv',
 './depresjon-dataset/control\\control_15.csv',
 './depresjon-dataset/control\\control_16.csv',
 './depresjon-dataset/control\\control_17.csv',
 './depresjon-dataset/control\\control_18.csv',
 './depresjon-dataset/control\\control_19.csv',
 './depresjon-dataset/control\\control_2.csv',
 './depresjon-dataset/control\\control_20.csv',
 './depresjon-dataset/control\\control_21.csv',
 './depresjon-dataset/control\\control_22.csv',
 './depresjon-dataset/control\\control_23.csv',
 './depresjon-dataset/control\\control_24.csv',
 './depresjon-dataset/control\\control_25.csv',
 './depresjon-dataset/control\\control_26.csv',
 './depresjon-dataset/control\\control_27.csv',
 './depresjon-dataset/control\\control_28.

In [18]:
li2 = []
for filename in stock_files_condition:
    #condtion = 'condition_'+ filename
    df = pd.read_csv(filename, parse_dates=["timestamp"], index_col="timestamp")
    li2.append(df)

In [19]:
li2[6]

,date,activity
timestamp,,
2003-11-25 09:00:00,2003-11-25,6
2003-11-25 09:01:00,2003-11-25,4
2003-11-25 09:02:00,2003-11-25,4
2003-11-25 09:03:00,2003-11-25,4
2003-11-25 09:04:00,2003-11-25,4
...,...,...
2003-12-10 19:58:00,2003-12-10,0
2003-12-10 19:59:00,2003-12-10,3
2003-12-10 20:00:00,2003-12-10,0


In [20]:
len(li2)

32

In [21]:
Final_Data2 =[]
#condtion = 'condition_'
for i, df in zip(range(1,len(li2)+1),li2):
 control = 'control_'+str(i)
 Final_Data2.append(FetureExtraction(df,control))

In [22]:
Final_Data2

[                           FId     fmean       fstd        fvar  fkurtosis  \
 timestamp                                                                    
 2003-03-18 15:00:00  control_1  2.999804  15.216270  231.534882  32.777312   
 2003-03-18 16:00:00  control_1 -0.673733   4.160133   17.306709  -0.432708   
 2003-03-18 17:00:00  control_1 -0.892069   3.704711   13.724887  -0.405841   
 2003-03-18 18:00:00  control_1 -0.387135   3.425349   11.733018  -0.292760   
 2003-03-18 19:00:00  control_1 -0.123317   3.301544   10.900190  -0.221257   
 ...                        ...       ...        ...         ...        ...   
 2003-04-23 08:00:00  control_1 -0.627490   3.114629    9.700911   0.093483   
 2003-04-23 09:00:00  control_1 -0.916646   3.765450   14.178616  -0.732157   
 2003-04-23 10:00:00  control_1 -0.539987   3.903713   15.238971  -0.762801   
 2003-04-23 11:00:00  control_1  0.987473   7.143813   51.034070  10.377489   
 2003-04-23 12:00:00  control_1  1.428150   8.620567

In [23]:
len(Final_Data2)

32

In [24]:
final_data2 = pd.concat(Final_Data2,axis=0)

In [25]:
final_data2

,FId,fmean,fstd,fvar,fkurtosis,fskewness,fquantile1,fquantile2,fquantile3,fquantile4,fquantile5,fquantile6,ftrimed_Mean,fcofvar,finvcofvar,fminimum,fmaxmimum,fmedian,fabsenergy,fIQR
timestamp,,,,,,,,,,,,,,,,,,,,
2003-03-18 15:00:00,control_1,2.999804,15.216270,231.534882,32.777312,5.147649,-13.539847,-7.195420,-3.192160,4.217417,16.296897,64.285372,0.843157,5.072421,0.197145,-16.227192,103.238838,0.743881,14200.487543,7.518789
2003-03-18 16:00:00,control_1,-0.673733,4.160133,17.306709,-0.432708,0.253298,-7.970607,-7.012162,-4.208167,2.221524,5.827366,8.033498,-0.778049,-6.174755,-0.161950,-8.415289,10.530621,-0.650247,1048.330767,6.460357
2003-03-18 17:00:00,control_1,-0.892069,3.704711,13.724887,-0.405841,0.054644,-8.588976,-6.395014,-3.383226,1.935216,4.885090,6.865162,-0.909607,-4.152943,-0.240793,-8.631674,7.752815,-1.088384,857.515546,5.425836
2003-03-18 18:00:00,control_1,-0.387135,3.425349,11.733018,-0.292760,-0.100203,-8.097199,-4.895464,-2.508841,2.083257,5.106821,6.281768,-0.401398,-8.847957,-0.113020,-9.020527,6.970806,-0.384166,701.240449,4.653210
2003-03-18 19:00:00,control_1,-0.123317,3.301544,10.900190,-0.221257,0.355532,-6.205761,-4.664622,-2.832335,1.994835,6.064482,7.401751,-0.263989,-26.772923,-0.037351,-6.484059,7.937971,-0.093340,644.023613,4.915149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003-12-01 08:00:00,control_32,0.115464,2.602271,6.771814,0.745205,0.509791,-5.165437,-3.855761,-1.451995,1.275324,5.696311,6.457918,-0.047877,22.537472,0.044371,-6.235356,6.480046,-0.055413,400.336938,2.757882
2003-12-01 09:00:00,control_32,-0.255316,2.083786,4.342165,-0.546511,-0.067340,-4.595262,-3.265648,-1.909770,1.311216,2.924195,3.511751,-0.241961,-8.161585,-0.122525,-5.498881,3.859524,-0.205695,260.098917,3.261649
2003-12-01 10:00:00,control_32,0.113865,2.904465,8.435918,0.841847,-0.178536,-6.713912,-3.990314,-1.911166,1.818663,5.404768,6.229738,0.084368,25.507874,0.039204,-8.956976,7.056883,0.185569,498.497104,3.951968


In [26]:
final_data2.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17021 entries, 2003-03-18 15:00:00 to 2003-12-01 12:00:00
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   FId           17021 non-null  object 
 1   fmean         17016 non-null  float64
 2   fstd          17016 non-null  float64
 3   fvar          17016 non-null  float64
 4   fkurtosis     17015 non-null  float64
 5   fskewness     17016 non-null  float64
 6   fquantile1    17016 non-null  float64
 7   fquantile2    17016 non-null  float64
 8   fquantile3    17016 non-null  float64
 9   fquantile4    17016 non-null  float64
 10  fquantile5    17016 non-null  float64
 11  fquantile6    17016 non-null  float64
 12  ftrimed_Mean  17016 non-null  float64
 13  fcofvar       17016 non-null  float64
 14  finvcofvar    17016 non-null  float64
 15  fminimum      17016 non-null  float64
 16  fmaxmimum     17016 non-null  float64
 17  fmedian       17016 non-null  floa

In [27]:
final_data2.insert(loc=1, column='Fclass', value='0')
final_data2.insert(loc=2, column='Fclass_str', value='c0')

In [28]:
final_data2

,FId,Fclass,Fclass_str,fmean,fstd,fvar,fkurtosis,fskewness,fquantile1,fquantile2,...,fquantile5,fquantile6,ftrimed_Mean,fcofvar,finvcofvar,fminimum,fmaxmimum,fmedian,fabsenergy,fIQR
timestamp,,,,,,,,,,,,,,,,,,,,,
2003-03-18 15:00:00,control_1,0,c0,2.999804,15.216270,231.534882,32.777312,5.147649,-13.539847,-7.195420,...,16.296897,64.285372,0.843157,5.072421,0.197145,-16.227192,103.238838,0.743881,14200.487543,7.518789
2003-03-18 16:00:00,control_1,0,c0,-0.673733,4.160133,17.306709,-0.432708,0.253298,-7.970607,-7.012162,...,5.827366,8.033498,-0.778049,-6.174755,-0.161950,-8.415289,10.530621,-0.650247,1048.330767,6.460357
2003-03-18 17:00:00,control_1,0,c0,-0.892069,3.704711,13.724887,-0.405841,0.054644,-8.588976,-6.395014,...,4.885090,6.865162,-0.909607,-4.152943,-0.240793,-8.631674,7.752815,-1.088384,857.515546,5.425836
2003-03-18 18:00:00,control_1,0,c0,-0.387135,3.425349,11.733018,-0.292760,-0.100203,-8.097199,-4.895464,...,5.106821,6.281768,-0.401398,-8.847957,-0.113020,-9.020527,6.970806,-0.384166,701.240449,4.653210
2003-03-18 19:00:00,control_1,0,c0,-0.123317,3.301544,10.900190,-0.221257,0.355532,-6.205761,-4.664622,...,6.064482,7.401751,-0.263989,-26.772923,-0.037351,-6.484059,7.937971,-0.093340,644.023613,4.915149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003-12-01 08:00:00,control_32,0,c0,0.115464,2.602271,6.771814,0.745205,0.509791,-5.165437,-3.855761,...,5.696311,6.457918,-0.047877,22.537472,0.044371,-6.235356,6.480046,-0.055413,400.336938,2.757882
2003-12-01 09:00:00,control_32,0,c0,-0.255316,2.083786,4.342165,-0.546511,-0.067340,-4.595262,-3.265648,...,2.924195,3.511751,-0.241961,-8.161585,-0.122525,-5.498881,3.859524,-0.205695,260.098917,3.261649
2003-12-01 10:00:00,control_32,0,c0,0.113865,2.904465,8.435918,0.841847,-0.178536,-6.713912,-3.990314,...,5.404768,6.229738,0.084368,25.507874,0.039204,-8.956976,7.056883,0.185569,498.497104,3.951968


In [30]:
final = [final_data1, final_data2]
final_data_Frq = pd.concat(final)

In [31]:
final_data_Frq

,FId,Fclass,Fclass_str,fmean,fstd,fvar,fkurtosis,fskewness,fquantile1,fquantile2,...,fquantile5,fquantile6,ftrimed_Mean,fcofvar,finvcofvar,fminimum,fmaxmimum,fmedian,fabsenergy,fIQR
timestamp,,,,,,,,,,,,,,,,,,,,,
2003-05-07 12:00:00,condition_1,1,c1,1.462541,13.286953,176.543115,30.587756,5.113729,-11.798675,-8.282023,...,6.339978,61.352314,-0.302254,9.084842,0.110073,-15.124664,86.675226,-0.943192,10544.385352,6.309015
2003-05-07 13:00:00,condition_1,1,c1,0.117842,3.158199,9.974221,-0.080292,-0.292813,-7.057130,-4.990460,...,4.960632,5.991059,0.189661,26.800245,0.037313,-8.768296,6.758724,0.254860,589.312248,4.329223
2003-05-07 14:00:00,condition_1,1,c1,0.172950,2.183673,4.768428,0.638762,0.036386,-4.454137,-3.851889,...,3.238580,5.828336,0.205902,12.626041,0.079201,-4.520817,6.185963,0.195132,283.131965,2.677316
2003-05-07 15:00:00,condition_1,1,c1,-0.000085,1.681192,2.826405,-0.473312,0.032258,-3.383892,-2.613928,...,2.597093,3.387807,-0.000755,-19667.292944,-0.000051,-3.857517,3.718487,0.033770,166.757919,2.394075
2003-05-07 16:00:00,condition_1,1,c1,-0.099009,1.515064,2.295417,-0.033818,0.007128,-3.496541,-2.345351,...,2.494810,3.078181,-0.115383,-15.302297,-0.065350,-3.905108,3.329462,-0.056935,136.017793,2.040137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003-12-01 08:00:00,control_32,0,c0,0.115464,2.602271,6.771814,0.745205,0.509791,-5.165437,-3.855761,...,5.696311,6.457918,-0.047877,22.537472,0.044371,-6.235356,6.480046,-0.055413,400.336938,2.757882
2003-12-01 09:00:00,control_32,0,c0,-0.255316,2.083786,4.342165,-0.546511,-0.067340,-4.595262,-3.265648,...,2.924195,3.511751,-0.241961,-8.161585,-0.122525,-5.498881,3.859524,-0.205695,260.098917,3.261649
2003-12-01 10:00:00,control_32,0,c0,0.113865,2.904465,8.435918,0.841847,-0.178536,-6.713912,-3.990314,...,5.404768,6.229738,0.084368,25.507874,0.039204,-8.956976,7.056883,0.185569,498.497104,3.951968


In [32]:
final_data_Frq.to_csv (r'final_data_Frq.csv', index = False, header=True)